In [1]:
import os
import time
import pandas as pd
import numpy as np
from scipy.stats import uniform
import matplotlib.pyplot as plt
import matplotlib_inline
import seaborn as sns
import mglearn

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

from sklearn.neural_network import MLPRegressor

# import helperfunctions
from ML_functions import fun_load_file, fun_preprocessing, fun_fit_tuning
from ML_functions import fun_convert_time
from ML_functions import fun_tuning_results, fun_scores

# Start time count
start_script = time.time()

# Assign string 'TSP' or 'CVRP' to the following variable to define the routing problem
routing_problem = 'TSP'

# Get the name of the folder and the file to store the final DataFrame
if (routing_problem == 'TSP'):
    folder = '01_TSP'
    file_name = 'tsp_instances_j_updated.xlsx'
elif (routing_problem == 'CVRP'):
    folder = '02_CVRP'
    file_name = 'cvrp_instances_j_updated.xlsx'

# Load data
data = fun_load_file(subfolder_path='..\\01_data\\' + folder, name=file_name)
X, y, train_data = fun_preprocessing(data)

# Create a train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# Save train size and create a dictionary to store the results
train_size = f'{int(np.round(100 * len(X_train)/len(X)))} %'
results_dict = {}

# **Neural Network - Multi Layer Perceptron**

In [ ]:
# Define a pipeline
pipe = make_pipeline(StandardScaler(),
                     MLPRegressor(activation='relu', solver='sgd', learning_rate='adaptive', batch_size='auto', 
                                  max_iter=1000, shuffle=True, verbose=False, random_state=0))

# Define parameter grid
param_grid = {'mlpregressor__hidden_layer_sizes': [(100, 100), (50, 50, 50), (100, 100, 100)],
              'mlpregressor__alpha': [0.05, 0.1, 0.5, 1]}

# Grid search
grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=3, 
                           scoring='neg_mean_absolute_percentage_error', verbose=True, n_jobs=-1)
fit_time = fun_fit_tuning(search_method=grid_search, X_train=X_train, y_train=y_train, file_name='NN_GS_best_params.pkl')

# Estimate model performance with cross validation on the train set (scoring: MAPE and RMSE)
model_results_dict = fun_scores(model=grid_search, X_train=X_train, y_train=y_train, cv=3)

# View grid search CV scores of all parameter combinations
results_df = fun_tuning_results(search_method=grid_search, search_space=param_grid)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
CV MAPE train data:  3.3866 %
CV RMSE train data:  0.6316

Best model / parameter combination:


{'mlp__alpha': 0.5, 'mlp__hidden_layer_sizes': (100, 100, 100)}

Cross validation scores of different parameter combinations:


,param_mlp__hidden_layer_sizes,param_mlp__alpha,mean_test_score,converted_mean_fit_time
0,"(100, 100, 100)",0.5,-0.033866,"17m, 0s"
1,"(100, 100)",0.1,-0.033881,"10m, 33s"
2,"(200, 200)",0.5,-0.033911,"24m, 14s"
3,"(100, 100)",0.5,-0.033961,"8m, 45s"
4,"(100, 100)",0.05,-0.034151,"12m, 11s"
5,"(50, 50, 50)",0.5,-0.034171,"10m, 26s"
6,"(50, 50, 50, 50)",0.5,-0.034214,"12m, 35s"
7,"(50, 50, 50, 50)",0.1,-0.034325,"14m, 21s"
8,"(50, 50, 50)",0.1,-0.034334,"8m, 11s"
9,"(200, 200)",0.1,-0.034445,"32m, 33s"


In [ ]:
# Define a pipeline
pipe = Pipeline([('scaler', StandardScaler()),
                 ('mlp', MLPRegressor(activation='relu', max_iter=1000, random_state=0))])

# Define search space
param_space = {'mlp__hidden_layer_sizes': [(10,), (50,), (100,), (50, 50)],
               'mlp__solver': Categorical(['sgd', 'adam']),
               'mlp__alpha': Real(0.0001, 0.05, prior='log-uniform'),
               'mlp__learning_rate': Categorical(['constant', 'adaptive'])}

# Bayesian optimization
bayes_search = BayesSearchCV(estimator=pipe, search_spaces=param_space, cv=3, 
                             scoring='neg_mean_absolute_percentage_error', n_iter=3, n_jobs=-1, random_state=0)
fit_time = fun_fit_tuning(search_method=bayes_search, X_train=X_train, y_train=y_train, file_name='NN_BS_best_params.pkl')

# Estimate model performance with cross validation on the train set (scoring: MAPE and RMSE)
model_results_dict = fun_scores(bayes_search, X_train, y_train, cv=3)

# View grid search CV scores of all parameter combinations
results_df = fun_tuning_results(grid_search, param_grid)

# **Compare Results**

In [ ]:
#display(pd.DataFrame(results_dict).sort_values(by='MAPE', axis=1))
print('Total script computation time:', fun_convert_time(start=start_script, end=time.time()))